# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import nltk
#from custom_transformer import StartingVerbExtractor
import pandas as pd
from sqlalchemy import create_engine, text
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer
import re
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.base import BaseEstimator,TransformerMixin
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
!pip install transformer

  Could not find a version that satisfies the requirement transformer (from versions: )
No matching distribution found for transformer


In [27]:
# load data from database
engine = create_engine('sqlite:///Message.db')
df = pd.read_sql_table('Message', engine) 
df=df.dropna()
X=df[['message']]
Y=df.drop(['message','original','genre','id'],axis=1)


In [28]:
import numpy as np
X=np.array(X,dtype).reshape(-1)
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       'shikarpur: Volunteers needed, food and medical help needed',
       'Dadu: Needs water, food, medicines. boats',
       'Umer kot: Sindh: Shelter and Camp needed as people are stranded due to flood:'], dtype=object)

### 2. Write a tokenization function to process your text data

In [5]:
from nltk.stem.porter import PorterStemmer
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    stemmed: list of strings. List containing normalized and stemmed word tokens
    """
    # Convert text to lowercase and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # Stem word tokens and remove stop words
    stemmer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    stemmed = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.multioutput import MultiOutputClassifier
pipeline = Pipeline([('vectorizer',CountVectorizer(tokenizer=tokenize)),
                     ('tfid', TfidfTransformer()),
                     ('clf',MultiOutputClassifier(DecisionTreeClassifier()))                        
                    ])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y)

pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y)
#print(X_train.shape,y_train.shape)
#X_train.shape[0] != y_train.shape[0]
#X_train.shape[1]==y_train.shape[1]
#y_train.shape

vectorizer=CountVectorizer()
tfid= TfidfTransformer()
clf=MultiOutputClassifier(estimator=DecisionTreeClassifier())
                      
    
X_new=vectorizer.fit_transform(X_train)
X_tfid=tfid.fit_transform(X_new)
#clf.fit(X_tfid,y_train)

X_tfid.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [9]:
print(X_train.shape)
print(y_train.shape)
pipeline.fit(X_train,y_train)

(7623,)
(7623, 36)


Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred[:,1]

NameError: name 'y_pred' is not defined

In [10]:
metrics_list_all=[]
category_names=y_test.columns
y_pred=pipeline.predict(X_test)
for col in range(y_test.shape[1]):
    tested=y_test.iloc[:,col].values.astype('int').astype('str')
    predicted=y_pred[:,col].astype('int').astype('str')
    accuracy = accuracy_score(tested, predicted)
    precision=precision_score(tested, predicted,average='micro')
    recall = recall_score(tested, predicted,average='micro')
    f_1 = f1_score(tested, predicted,average='micro')
    metrics_list=[accuracy,precision,recall,f_1]
    metrics_list_all.append(metrics_list)
metrics_df=pd.DataFrame(metrics_list_all,index=category_names,columns=["Accuracy","Precision","Recall","F_1"])
print(metrics_df)

                        Accuracy  Precision    Recall       F_1
related                 0.562943   0.562943  0.562943  0.562943
request                 0.577891   0.577891  0.577891  0.577891
offer                   0.999213   0.999213  0.999213  0.999213
aid_related             0.561369   0.561369  0.561369  0.561369
medical_help            0.898505   0.898505  0.898505  0.898505
medical_products        0.939418   0.939418  0.939418  0.939418
search_and_rescue       0.964201   0.964201  0.964201  0.964201
security                0.973249   0.973249  0.973249  0.973249
military                0.994886   0.994886  0.994886  0.994886
child_alone             1.000000   1.000000  1.000000  1.000000
water                   0.872935   0.872935  0.872935  0.872935
food                    0.763965   0.763965  0.763965  0.763965
shelter                 0.830055   0.830055  0.830055  0.830055
clothing                0.981904   0.981904  0.981904  0.981904
money                   0.981511   0.981

In [60]:
y_pred=(pipeline.predict(X_test))
y_pred[:,0]

pipeline

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [11]:
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
y_labels=y_test.columns
x=0
for column in y_test.columns:  
    tested=y_test.iloc[:,col].values.astype('int').astype('str')
    predicted=y_pred[:,col].astype('int').astype('str')
    
    print(classification_report(tested, predicted))
    x=x+1

             precision    recall  f1-score   support

          0       0.66      0.69      0.68      1683
          1       0.34      0.32      0.33       859

avg / total       0.55      0.56      0.56      2542

             precision    recall  f1-score   support

          0       0.66      0.69      0.68      1683
          1       0.34      0.32      0.33       859

avg / total       0.55      0.56      0.56      2542

             precision    recall  f1-score   support

          0       0.66      0.69      0.68      1683
          1       0.34      0.32      0.33       859

avg / total       0.55      0.56      0.56      2542

             precision    recall  f1-score   support

          0       0.66      0.69      0.68      1683
          1       0.34      0.32      0.33       859

avg / total       0.55      0.56      0.56      2542

             precision    recall  f1-score   support

          0       0.66      0.69      0.68      1683
          1       0.34      0.32 

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
sorted(pipeline.get_params().keys())

['clf',
 'clf__estimator',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__presort',
 'clf__estimator__random_state',
 'clf__estimator__splitter',
 'clf__n_jobs',
 'memory',
 'steps',
 'tfid',
 'tfid__norm',
 'tfid__smooth_idf',
 'tfid__sublinear_tf',
 'tfid__use_idf',
 'vectorizer',
 'vectorizer__analyzer',
 'vectorizer__binary',
 'vectorizer__decode_error',
 'vectorizer__dtype',
 'vectorizer__encoding',
 'vectorizer__input',
 'vectorizer__lowercase',
 'vectorizer__max_df',
 'vectorizer__max_features',
 'vectorizer__min_df',
 'vectorizer__ngram_range',
 'vectorizer__preprocessor',
 'vectorizer__stop_words',
 'vectorizer__strip_accents',
 'vectorizer__token_pa

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {
             #  'vect__stop_words': ('english', None),
                'clf__estimator__criterion':['gini','entropy'],
                'clf__estimator__max_features': ['auto','sqrt','log2'],
                'clf__estimator__min_samples_split': [2, 5]
             }
cv = GridSearchCV(pipeline,param_grid=parameters)
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_features': ['auto', 'sqrt', 'log2'], 'clf__estimator__min_samples_split': [2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
(y_test==y_pred).mean().mean()

0.90734548474517007

In [13]:
metrics_list_all=[]
category_names=y_test.columns
y_pred=cv.predict(X_test)
for col in range(y_test.shape[1]):
    tested=y_test.iloc[:,col].values.astype('int').astype('str')
    predicted=y_pred[:,col].astype('int').astype('str')
    accuracy = accuracy_score(tested, predicted)
    precision=precision_score(tested, predicted,average='micro')
    recall = recall_score(tested, predicted,average='micro')
    f_1 = f1_score(tested, predicted,average='micro')
    metrics_list=[accuracy,precision,recall,f_1]
    metrics_list_all.append(metrics_list)
metrics_df_new=pd.DataFrame(metrics_list_all,index=category_names,columns=["Accuracy_new","Precision_new","Recall_new","F_1_new"])
print(metrics_df_new)

                        Accuracy_new  Precision_new  Recall_new   F_1_new
related                     0.558615       0.558615    0.558615  0.558615
request                     0.582219       0.582219    0.582219  0.582219
offer                       0.998820       0.998820    0.998820  0.998820
aid_related                 0.557042       0.557042    0.557042  0.557042
medical_help                0.910307       0.910307    0.910307  0.910307
medical_products            0.951613       0.951613    0.951613  0.951613
search_and_rescue           0.972069       0.972069    0.972069  0.972069
security                    0.980724       0.980724    0.980724  0.980724
military                    0.996459       0.996459    0.996459  0.996459
child_alone                 1.000000       1.000000    1.000000  1.000000
water                       0.884343       0.884343    0.884343  0.884343
food                        0.783242       0.783242    0.783242  0.783242
shelter                     0.846184  

In [87]:
compare_per_DF=pd.concat([metrics_df_new,metrics_df],axis=1)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
y_labels=y_test.columns
x=0
upd_y_pred= upd_pipeline.predict(X_test)
for column in y_test.columns:  
    tested=y_test.iloc[:,col].values.astype('int').astype('str')
    predicted=upd_y_pred[:,col].astype('int').astype('str')
    
    print(classification_report(tested, predicted))
    x=x+1
    

NameError: name 'upd_pipeline' is not defined

In [16]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            print(pos_tags)
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
            return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [23]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.pipeline import FeatureUnion
#from custom_transformer import StartingVerbExtractor
upd_pipeline = Pipeline([
        ('features', FeatureUnion ([
            
            ('text_pipeline', Pipeline ([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
                                        ])),      
             ('starting_verb', StartingVerbExtractor())   
                                    ])),        
        
        ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier))
        ])


upd_pipeline.fit(X_train, y_train)

    


[('need', 'NN'), ('aid', 'NN'), ('fontamara', 'NN'), ('husband', 'NN')]
[('peopl', 'JJ'), ('area', 'NN'), ('laplain', 'NN'), ('marin', 'NNS'), ('need', 'VBP'), ('electr', 'JJ'), ('sinc', 'NN'), ('yesterday', 'NN'), ('give', 'VB')]
[('victim', 'NN'), ('southern', 'JJ'), ('depart', 'NN'), ('never', 'RB'), ('ever', 'RB'), ('get', 'VB'), ('help', 'NN')]
[('good', 'JJ'), ('even', 'RB'), ('champ', 'VBP'), ('de', 'FW'), ('mar', 'FW'), ('st', 'NN')]
[('hour', 'NN'), ('hous', 'JJ'), ('destroy', 'NN'), ('spend', 'NN'), ('entir', 'JJ'), ('life', 'NN'), ('sleep', 'JJ'), ('outsid', 'NN'), ('fine', 'JJ'), ('rain', 'NN'), ('serein', 'NN')]
[('messg', 'JJ'), ('framgment', 'NN'), ('say', 'VBP'), ('aunt', 'NN'), ('ny', 'JJ'), ('ask', 'NN'), ('mvissa', 'NN'), ('cananda', 'NN'), ('get', 'NN')]
[('excus', 'NN'), ('dear', 'JJ'), ('account', 'NN'), ('empti', 'JJ'), ('keep', 'VB'), ('call', 'NN')]
[('ong', 'NNS'), ('say', 'VBP'), ('treat', 'JJ'), ('water', 'NN'), ('akwatab', 'NN'), ('littl', 'NN'), ('clorox',

TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

### 9. Export your model as a pickle file

In [35]:
pickle.dump(pipeline,'/home/workspace/.')

NameError: name 'pickle' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [34]:
pwd

'/home/workspace'